<a href="https://www.kaggle.com/code/csbenk/ollama-issues-classifier?scriptVersionId=220299334" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# Setup

In [ ]:
!pip install datasets evaluate transformers[sentencepiece]
!apt install git-lfs

In [2]:
!git config --global user.email "csbhaskar95@gmail.com"
!git config --global user.name "25b3nk"

In [3]:
from huggingface_hub import notebook_login

notebook_login()

# Load dataset and filter the data to get ready for training

In [4]:
from datasets import load_dataset

remote_dataset = load_dataset("25b3nk/ollama-github-issues")
remote_dataset

README.md:   0%|          | 0.00/5.84k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/6.78M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/1.82M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/6860 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1716 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['url', 'repository_url', 'labels_url', 'comments_url', 'events_url', 'html_url', 'id', 'node_id', 'number', 'title', 'user', 'labels', 'state', 'locked', 'assignee', 'assignees', 'milestone', 'comments', 'created_at', 'updated_at', 'closed_at', 'author_association', 'sub_issues_summary', 'active_lock_reason', 'draft', 'pull_request', 'body', 'closed_by', 'reactions', 'timeline_url', 'performed_via_github_app', 'state_reason', 'is_pull_request'],
        num_rows: 6860
    })
    test: Dataset({
        features: ['url', 'repository_url', 'labels_url', 'comments_url', 'events_url', 'html_url', 'id', 'node_id', 'number', 'title', 'user', 'labels', 'state', 'locked', 'assignee', 'assignees', 'milestone', 'comments', 'created_at', 'updated_at', 'closed_at', 'author_association', 'sub_issues_summary', 'active_lock_reason', 'draft', 'pull_request', 'body', 'closed_by', 'reactions', 'timeline_url', 'performed_via_github_app', 'state_reason

In [5]:
filtered_dataset = remote_dataset.filter(lambda x: x["is_pull_request"] == False).filter(lambda x: x["body"] is not None)

Filter:   0%|          | 0/6860 [00:00<?, ? examples/s]

Filter:   0%|          | 0/1716 [00:00<?, ? examples/s]

Filter:   0%|          | 0/4553 [00:00<?, ? examples/s]

Filter:   0%|          | 0/1168 [00:00<?, ? examples/s]

In [6]:
all_labels = set()
for labels in filtered_dataset["train"]["labels"]:
  for label in labels:
    all_labels.add(label["name"])

In [7]:
label2id = {label: i for i, label in enumerate(all_labels)}
id2label = {i: label for i, label in enumerate(all_labels)}

In [8]:
import numpy as np

def encode_labels(example):
    labels = example['labels']  # Split string into individual labels
    label_ids = [label2id[label["name"]] for label in labels if label["name"] in label2id]

    # Create a multi-hot vector with the length of all unique labels
    multi_hot = np.zeros(len(all_labels), dtype=int)
    multi_hot[label_ids] = 1
    example['multi_hot_labels'] = multi_hot.tolist()  # Convert back to a list to save in dataset
    example['label_ids'] = label_ids
    return example

In [9]:
encoded_dataset = filtered_dataset.map(encode_labels)

Map:   0%|          | 0/4484 [00:00<?, ? examples/s]

Map:   0%|          | 0/1152 [00:00<?, ? examples/s]

# Tokenize the dataset

In [10]:
from transformers import AutoTokenizer

checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [12]:
encoded_dataset["train"].features["body"]
tokenizer(encoded_dataset["train"][0]["body"], truncation=True)

{'input_ids': [101, 1045, 4384, 2008, 1996, 19330, 10278, 2050, 2544, 12057, 2004, 8946, 2121, 11661, 2038, 2042, 7172, 2000, 1014, 1012, 1015, 1012, 2654, 1998, 2947, 2323, 2448, 2732, 16044, 2099, 2475, 1998, 19073, 4275, 1011, 1045, 2572, 2145, 2025, 2383, 6735, 2770, 2216, 1010, 19330, 10278, 2050, 2074, 19119, 1012, 1012, 1012, 2572, 1045, 4394, 2242, 1029, 16770, 1024, 1013, 1013, 19351, 8428, 1012, 4012, 1013, 2632, 3501, 12881, 13213, 2629, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [11]:
def tokenize_github_issues(examples):
  return tokenizer(examples["body"], truncation=True, padding=True)

In [12]:
tokenized_datasets = encoded_dataset.map(tokenize_github_issues, batched=True)

Map:   0%|          | 0/4484 [00:00<?, ? examples/s]

Map:   0%|          | 0/1152 [00:00<?, ? examples/s]

In [13]:
tokenized_datasets = tokenized_datasets.remove_columns(['url', 'repository_url', 'labels_url', 'comments_url', 'events_url', 'html_url', 'id', 'node_id', 'number', 'title', 'user', 'labels', 'state', 'locked', 'assignee', 'assignees', 'milestone', 'comments', 'created_at', 'updated_at', 'closed_at', 'author_association', 'sub_issues_summary', 'active_lock_reason', 'draft', 'pull_request', 'body', 'closed_by', 'reactions', 'timeline_url', 'performed_via_github_app', 'state_reason', 'is_pull_request'])
tokenized_datasets = tokenized_datasets.rename_column("multi_hot_labels", "labels")

In [16]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['labels', 'label_ids', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 4484
    })
    test: Dataset({
        features: ['labels', 'label_ids', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 1152
    })
})

In [14]:
import torch

tokenized_datasets.set_format('torch', columns=['input_ids', 'attention_mask', 'labels'])
# tokenized_datasets = tokenized_datasets.map(lambda x: {'labels': torch.FloatTensor(x['labels'])})

# Setup training params

In [15]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [16]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(
    tokenized_datasets["train"], shuffle=True, batch_size=32, collate_fn=data_collator
)

eval_dataloader = DataLoader(
    tokenized_datasets["test"], batch_size=32, collate_fn=data_collator
)

In [19]:
from transformers import AutoModelForSequenceClassification
checkpoint = "25b3nk/ollama-issues-classifier"
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=len(all_labels),id2label=id2label, label2id=label2id, problem_type="multi_label_classification")

config.json:   0%|          | 0.00/2.19k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

In [21]:
# Testing
for batch in train_dataloader:
  val = {k: v.shape for k, v in batch.items()}
  print(f"{val}")
  batch['labels'] = batch['labels'].float()
  outputs = model(**batch)
  print(outputs.loss, outputs.logits.shape)
  break

{'labels': torch.Size([32, 35]), 'input_ids': torch.Size([32, 512]), 'attention_mask': torch.Size([32, 512])}
tensor(0.7170, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>) torch.Size([32, 35])


In [32]:
import torch
from transformers import AdamW
from sklearn.metrics import accuracy_score, f1_score

import os

# Directory to save the checkpoint
checkpoint_dir = "./checkpoint_dir"
os.makedirs(checkpoint_dir, exist_ok=True)

# Save model, optimizer, and epoch number
def save_checkpoint(model, optimizer, epoch, step):
    model.save_pretrained(checkpoint_dir)
    tokenizer.save_pretrained(checkpoint_dir)
    torch.save({
        'epoch': epoch,
        'step': step,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
    }, os.path.join(checkpoint_dir, 'checkpoint.pt'))
    print(f"Saved to {os.path.join(checkpoint_dir, 'checkpoint.pt')}")

def load_checkpoint(model, optimizer, checkpoint_dir):
  ck_path = os.path.join(checkpoint_dir, 'checkpoint.pt')
  if not os.path.exists(ck_path):
    return model, optimizer, 0, 0
  checkpoint = torch.load(ck_path)
  model.load_state_dict(checkpoint['model_state_dict'])
  optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
  start_epoch = checkpoint['epoch']
  start_step = checkpoint['step']
  return model, optimizer, start_epoch, start_step

In [23]:
# Move the model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Optimizer
optimizer = AdamW(model.parameters(), lr=5e-5)

# Custom loss function (BCEWithLogitsLoss for multi-label classification)
loss_fn = torch.nn.BCEWithLogitsLoss()

# Function for calculating metrics
def compute_metrics(preds, labels):
    sigmoid_preds = torch.sigmoid(preds).cpu().numpy()
    sigmoid_preds = (sigmoid_preds > 0.5).astype(int)  # Convert logits to 0 or 1

    labels = labels.cpu().numpy()
    acc = accuracy_score(labels, sigmoid_preds)
    f1 = f1_score(labels, sigmoid_preds, average='micro')

    return {"accuracy": acc, "f1": f1}

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [37]:
from tqdm import tqdm

def train(model, optimizer, epochs=50, N = 100):
  # Reload model and optimizer state from checkpoint
  model, optimizer, start_epoch, start_step = load_checkpoint(model, optimizer, checkpoint_dir)
  current_acc = 0.58
  current_f1 = 0.67
  # Continue saving checkpoints or additional logic as required

  # Resume training from saved state
  for epoch in range(start_epoch, epochs):
      model.train()
      for step, batch in tqdm(enumerate(train_dataloader, start=start_step), total=len(train_dataloader)):
          optimizer.zero_grad()

          # Move batch to the appropriate device
          input_ids = batch['input_ids'].to(device)
          attention_mask = batch['attention_mask'].to(device)
          labels = batch['labels'].float().to(device)

          # Forward pass
          outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
          loss = outputs.loss

          loss.backward()
          optimizer.step()



      print(f"Epoch {epoch + 1}/{epochs} completed.")
      # Evaluate the model after each epoch
      model.eval()
      preds, true_labels = [], []
      with torch.no_grad():
          for batch in eval_dataloader:
              input_ids = batch['input_ids'].to(device)
              attention_mask = batch['attention_mask'].to(device)
              labels = batch['labels'].float().to(device)

              outputs = model(input_ids=input_ids, attention_mask=attention_mask)
              preds.append(outputs.logits)
              true_labels.append(labels)

      # if epoch % 20 == 0:
      #     model.push_to_hub("ollama-issues-classifier")
      #     tokenizer.push_to_hub("ollama-issues-classifier")

      # Save checkpoint
      save_checkpoint(model, optimizer, epoch, step)
      print(f"Checkpoint saved at step {step} of epoch {epoch}.")
      preds = torch.cat(preds)
      true_labels = torch.cat(true_labels)
      metrics = compute_metrics(preds, true_labels)

      print(f"Validation metrics: {metrics}")
      if metrics["accuracy"] > current_acc and metrics["f1"] > current_f1:
          print(f"Pushing the model as the new eval metrics is better than older metrics. earlier_acc: {current_acc} earlier_f1: {current_f1}")
          current_acc = metrics["accuracy"]
          current_f1 = metrics["f1"]
          model.push_to_hub("ollama-issues-classifier")
          tokenizer.push_to_hub("ollama-issues-classifier")

  model.push_to_hub("ollama-issues-classifier")
  tokenizer.push_to_hub("ollama-issues-classifier")

In [ ]:
train(model, optimizer, epochs=50)

<ipython-input-32-965ed69fd6ae>:27: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(ck_path)
100%|██████████| 141/141 [03:52<00:00,  1.65s/it]


Epoch 2/50 completed.
Saved to ./checkpoint_dir/checkpoint.pt
Checkpoint saved at step 140 of epoch 1.
Validation metrics: {'accuracy': 0.5642361111111112, 'f1': 0.6587301587301587}


100%|██████████| 141/141 [03:52<00:00,  1.65s/it]


Epoch 3/50 completed.
Saved to ./checkpoint_dir/checkpoint.pt
Checkpoint saved at step 140 of epoch 2.
Validation metrics: {'accuracy': 0.5807291666666666, 'f1': 0.6626626626626626}


100%|██████████| 141/141 [03:52<00:00,  1.65s/it]


...

100%|██████████| 141/141 [03:53<00:00,  1.66s/it]


Epoch 43/50 completed.
Saved to ./checkpoint_dir/checkpoint.pt
Checkpoint saved at step 140 of epoch 42.
Validation metrics: {'accuracy': 0.5095486111111112, 'f1': 0.6402910413824466}


100%|██████████| 141/141 [03:53<00:00,  1.66s/it]


Epoch 44/50 completed.
Saved to ./checkpoint_dir/checkpoint.pt
Checkpoint saved at step 140 of epoch 43.
Validation metrics: {'accuracy': 0.5190972222222222, 'f1': 0.6448979591836735}


 28%|██▊       | 39/141 [01:04<02:49,  1.67s/it]